In [1]:
import bw2data, bw2calc, bw2io
from edges import EdgeLCIA, get_available_methods
from bw2analyzer.contribution import ContributionAnalysis

In [2]:
bw2data.projects.set_current("ecoinvent-3.10-cutoff")
act = bw2data.Database("ecoinvent-3.10.1-cutoff").search("electricity production, medium voltage")[-1]
#act = bw2data.Database("ecoinvent-3.10.1-cutoff").random()
act

'market for electricity, medium voltage' (kilowatt hour, SS, None)

In [3]:
method = ('GeoPolRisk', 'paired', '2024')
LCA = EdgeLCIA({act: 1}, method)
LCA.lci()
LCA.map_exchanges()
LCA.map_aggregate_locations()
LCA.map_dynamic_locations()
LCA.map_remaining_locations_to_global()
LCA.evaluate_cfs()
LCA.lcia()

Mapping 155115 exchanges...


Mapping exchanges: 100%|██████████| 51932/51932 [00:14<00:00, 3695.84it/s]


Processed edges: 161
Handling static regions...


Processing static groups (pass 1): 0it [00:00, ?it/s]


The length of Entered edge list is: %s 0


Processing static groups (pass 2): 0it [00:00, ?it/s]
Processing static groups (pass 1): 100%|██████████| 6/6 [00:00<00:00, 1199.97it/s]


The length of Entered edge list is: %s 38433


ValueError: too many values to unpack (expected 4)

In [ ]:
df = LCA.generate_cf_table()

In [ ]:
len(df)

In [ ]:
df

In [ ]:
df.groupby(["supplier location", "consumer location"])["impact"].mean()

In [ ]:
LCA.statistics()

In [ ]:
LCA.score

In [ ]:
df = LCA.generate_cf_table()

In [ ]:
df

In [ ]:
LCA.statistics()

In [ ]:
import matplotlib.pyplot as plt
methods = [('GeoPolRisk', 'paired', '2024'),]
bw2data.projects.set_current("ecoinvent-3.10-cutoff")
act = bw2data.Database("ecoinvent-3.10.1-cutoff").random()
print(act)
for method in methods:
    LCA = EdgeLCIA({act: 1}, method)
    LCA.lci()
    LCA.map_exchanges()
    LCA.map_aggregate_locations()
    LCA.map_dynamic_locations()
    LCA.map_remaining_locations_to_global()
    LCA.evaluate_cfs()
    LCA.lcia()
    df = LCA.generate_cf_table()

    cutoff = 0.01

    # remove rows for which "amount" is below the cutoff
    total = df["impact"].sum()
    df = df[df["impact"] > cutoff * total]

    # Plotting
    fig, axes = plt.subplots(3, 2, figsize=(10, 12))

    # 1. Bar plot of "amount" by "supplier name"
    df.groupby("supplier name")["impact"].sum().plot(kind="bar", ax=axes[0, 0])
    axes[0, 0].set_title("Impact by Supplier Name")
    axes[0, 0].set_ylabel("Impact")
    axes[0, 0].tick_params(axis="x", rotation=45)

    # 2. Bar plot of "amount" by "consumer name"
    df.groupby("consumer name")["impact"].sum().plot(kind="bar", ax=axes[0, 1])
    axes[0, 1].set_title("Impact by Consumer Name")
    axes[0, 1].set_ylabel("Impact")
    axes[0, 1].set_xticklabels(
        [label.get_text().replace(", ", "\n") for label in axes[0, 1].get_xticklabels()],
        rotation=45,
        ha="right",
    )

    # 3. Bar plot of "impact" by "consumer location"
    df.groupby("consumer location")["impact"].sum().plot(kind="bar", ax=axes[1, 0])
    axes[1, 0].set_title("Impact by Consumer Location")
    axes[1, 0].set_ylabel("Impact")
    axes[1, 0].tick_params(axis="x", rotation=45)


    # 3. Bar plot of "impact" by "supplier location"
    df.groupby("supplier location")["impact"].sum().plot(kind="bar", ax=axes[1, 1])
    axes[1, 1].set_title("Impact by Supplier Location")
    axes[1, 1].set_ylabel("Impact")
    axes[1, 1].tick_params(axis="x", rotation=45)


    # 4. Stacked bar plot for "amount" by "consumer name" and "consumer location"
    df_grouped = df.groupby(["supplier name", "supplier location"])["impact"].sum().unstack()
    df_grouped.plot(kind="bar", stacked=True, ax=axes[2, 0])
    axes[2, 0].set_title("Impact by Supplier Name & Location (Stacked)")
    axes[2, 0].set_ylabel("Impact")
    axes[2, 0].set_xticklabels(
        [label.get_text().replace(", ", "\n") for label in axes[2, 0].get_xticklabels()],
        rotation=45,
        ha="right",
    )
    axes[2, 0].legend(title="Supplier Location", loc="center left", ncol=2)

    # 5. Bar plot of average "CF" by "consumer location"
    df.groupby(["supplier name", "supplier location"])["CF"].mean().plot(kind="bar", ax=axes[2, 1])
    axes[2, 1].set_title("Average CF by Supplier Location")
    axes[2, 1].set_ylabel("CF")
    axes[2, 1].tick_params(axis="x", rotation=45)


    plt.tight_layout()
    plt.show()

    

  #LCA.score

In [ ]:
df.sort_values(by="impact", ascending=False).loc[:, "supplier name"]

In [ ]:
df.loc[df["consumer name"] == "soybean production"]

In [ ]:
df.sort_values("amount", ascending=False)

In [ ]:
df["CF"].unique()

In [ ]:
df.groupby(["supplier name", "consumer location"])["CF"].mean()

In [ ]:
list(LCA.demand.keys())[0]

In [ ]:
from constructive_geometries import geomatcher
geo = geomatcher.Geomatcher()

geo.within("PG", biggest_first=False)

In [ ]:
geo.contained("PG", biggest_first=False)

In [ ]:
set([x["supplier"]["name"] for x in LCA.ignored_method_exchanges])

In [ ]:
LCA.generate_cf_table()

In [ ]:
LCA.ignored_locations

In [ ]:
method = ('AWARE 1.2c', 'Country', 'mixed', 'yearly')
convmethod = (
  # 'ecoinvent-3.10',
  'EF v3.1 EN15804',
  'water use',
  'user deprivation potential (deprivation-weighted water consumption)')

In [ ]:
import bw2calc

convLCA = bw2calc.LCA({act: 1}, convmethod)
convLCA.lci()
convLCA.lcia()
convLCA.score